In [61]:
import pandas.io.sql
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

Подключаемся к базе данных

In [2]:
import pyodbc 
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=LAPTOP-KIBD4RBV;'
                      'Database=AdventureWorks2017;'
                      'Trusted_Connection=yes;')

cursor = conn.cursor()

In [3]:
sql = 'SELECT * FROM AdventureWorks2017.Sales.SalesOrderHeader'

Сохраняем данные

In [35]:
df = pandas.io.sql.read_sql_query(sql, conn)

In [36]:
df.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,...,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43659,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43659,PO522145787,10-4020-000676,...,16281.0,105041Vi84182,NaN,20565.6206,1971.5149,616.0984,23153.2339,None,79B65321-39CA-4115-9CBA-8FE0903E12E6,2011-06-07
1,43660,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,...,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
2,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,...,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
3,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,...,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
4,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,...,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31465 entries, 0 to 31464
Data columns (total 26 columns):
SalesOrderID              31465 non-null int64
RevisionNumber            31465 non-null int64
OrderDate                 31465 non-null datetime64[ns]
DueDate                   31465 non-null datetime64[ns]
ShipDate                  31465 non-null datetime64[ns]
Status                    31465 non-null int64
OnlineOrderFlag           31465 non-null bool
SalesOrderNumber          31465 non-null object
PurchaseOrderNumber       3806 non-null object
AccountNumber             31465 non-null object
CustomerID                31465 non-null int64
SalesPersonID             3806 non-null float64
TerritoryID               31465 non-null int64
BillToAddressID           31465 non-null int64
ShipToAddressID           31465 non-null int64
ShipMethodID              31465 non-null int64
CreditCardID              30334 non-null float64
CreditCardApprovalCode    30334 non-null object
CurrencyRateI

Возьмем OnlineOrderFlag как целевую переменную. Будем определять закажет ли клиент онлайн или через продавца

*0 = Order placed by sales person. 1 = Order placed online by customer*

In [38]:
df.OnlineOrderFlag.value_counts()

True     27659
False     3806
Name: OnlineOrderFlag, dtype: int64

Все признаки нам не нужны, выберем несколько для работы

In [51]:
df_fa = df[['TerritoryID', 'TotalDue', 'OnlineOrderFlag','Status','SalesPersonID']]

In [52]:
df_fa.head()

,TerritoryID,TotalDue,OnlineOrderFlag,Status,SalesPersonID
0,5,23153.2339,False,5,279.0
1,5,1457.3288,False,5,279.0
2,6,36865.8012,False,5,282.0
3,6,32474.9324,False,5,282.0
4,4,472.3108,False,5,276.0


In [54]:
df_terr = pd.get_dummies(df_fa['TerritoryID'], drop_first=True, prefix='Territory')
df_fa = pd.concat([df_fa.drop('TerritoryID',axis=1),df_terr], axis=1)

In [55]:
df_Status = pd.get_dummies(df_fa['Status'], drop_first=True, prefix='Status')
df_fa = pd.concat([df_fa.drop('Status',axis=1),df_Status], axis=1)

In [56]:
df_SP = pd.get_dummies(df_fa['SalesPersonID'], drop_first=True, prefix='Status')
df_fa = pd.concat([df_fa.drop('SalesPersonID',axis=1),df_SP], axis=1)

In [58]:
def data_split_train_test(data, test_size=0.3):
    X_train, X_test, y_train, y_test = train_test_split(
    data.drop(labels=['OnlineOrderFlag'], axis=1),
    data['OnlineOrderFlag'],
    test_size=test_size,
    stratify=data['OnlineOrderFlag'],
    random_state=0)

    print("Train_shape:", X_train.shape, "Test shape:", X_test.shape)
    
    return X_train, X_test, y_train, y_test

In [59]:
X_train, X_test, y_train, y_test = data_split_train_test(df_fa)

Train_shape: (22025, 26) Test shape: (9440, 26)


In [63]:
log_reg = LogisticRegression()

log_reg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [64]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def print_quality_report(prediction, actual):
    Accuracy = accuracy_score(prediction, actual)
    Precision = precision_score(prediction, actual)
    Recall = recall_score(prediction, actual)
    F1 = f1_score(prediction, actual)
        
    quality_list = [Accuracy, Precision, Recall, F1]
    
    return quality_list

In [65]:
quality_list_lr = print_quality_report(log_reg.predict(X_train), y_train)

df_quality = pd.DataFrame(
    [quality_list_lr],
    columns=["Accuracy", "Precision", "Recall"," F1 score"],
    index=["Logistic Regression Train"])

df_quality.loc["Logistic Regression Test", :] = print_quality_report(log_reg.predict(X_test), y_test)

df_quality.round(3)

,Accuracy,Precision,Recall,F1 score
Logistic Regression Train,0.997,1.0,0.997,0.999
Logistic Regression Test,0.997,1.0,0.996,0.998


In [66]:
conn.close()

Мы получили очень хорошие результаты по прогнозу будет ли клиент заказывать онлайн